In [ ]:
import pandas as pd
import numpy as np
import os
import pickle
from recomendation_model.base_model import LightGBMRanker

os.chdir("c://Users//gufer//OneDrive//Documentos//FIAP//Fase_05//ML_Engineer_Datathon/")
# Carrega os dados de treino a partir dos arquivos Parquet
X_train = pd.read_parquet("data/train/X_train.parquet")
y_train = pd.read_parquet("data/train/y_train.parquet")
X_train_full = pd.read_parquet("data/train/X_train_full.parquet")
group_train = pd.read_parquet("data/train/group_train.parquet")
# Note que para o parâmetro 'group' do LightGBM, passamos os valores da coluna 'groupCount'
model = LightGBMRanker()
model.train(X_train.values, y_train.values.ravel(), group_train["groupCount"].values)

# Salve o modelo em um arquivo pickle
import pickle
with open("lightgbm_ranker.pkl", "wb") as f:
    pickle.dump(model, f)

print("Modelo treinado e salvo em 'lightgbm_ranker.pkl'")
with open("lightgbm_ranker.pkl", "rb") as f:
    model = pickle.load(f)

print("Modelo carregado com sucesso!")

Modelo treinado e salvo em 'lightgbm_ranker.pkl'
Modelo carregado com sucesso!


In [3]:
CLIENT_FEATURES = [
    'userTypeFreq', 
    'isWeekend', 
    'dayPeriodFreq',
]

NEWS_FEATURES = [
    'relLocalState', 
    'relLocalRegion',
    'relThemeMain', 
    'relThemeSub',  
    'localStateFreq', 
    'localRegionFreq', 
    'themeMainFreq',
    'themeSubFreq',
]

client_features = X_train[CLIENT_FEATURES]
news_features = X_train[NEWS_FEATURES] 

In [5]:
client_features

,userTypeFreq,isWeekend,dayPeriodFreq
0,0.476879,False,0.397835
1,0.523121,False,0.144567
2,0.476879,False,0.135343
3,0.476879,True,0.397835
4,0.476879,False,0.322255
...,...,...,...
520731,0.476879,False,0.322255
520732,0.476879,False,0.322255
520733,0.523121,False,0.144567
520734,0.476879,False,0.397835


In [6]:
news_features

,relLocalState,relLocalRegion,relThemeMain,relThemeSub,localStateFreq,localRegionFreq,themeMainFreq,themeSubFreq
0,NaN,NaN,0.181818,NaN,NaN,NaN,0.116344,NaN
1,0.093923,0.093923,NaN,NaN,0.035214,0.035214,NaN,NaN
2,0.275000,0.050000,NaN,NaN,0.267608,0.018291,NaN,NaN
3,NaN,NaN,0.151515,NaN,NaN,NaN,0.146418,NaN
4,0.118852,0.004098,NaN,NaN,0.267608,0.004950,NaN,NaN
...,...,...,...,...,...,...,...,...
520731,0.798450,0.086822,NaN,NaN,0.267608,0.007533,NaN,NaN
520732,0.193548,0.064516,NaN,NaN,0.061350,0.024310,NaN,NaN
520733,0.007092,0.007092,NaN,NaN,0.018322,0.018322,NaN,NaN
520734,NaN,NaN,0.018182,0.018182,NaN,NaN,0.011221,0.014284


In [7]:
model?

Type:           LightGBMRanker
String form:    <recomendation_model.base_model.LightGBMRanker object at 0x000001E626146790>
File:           c:\users\gufer\onedrive\documentos\fiap\fase_05\ml_engineer_datathon\src\recomendation_model\base_model.py
Docstring:     
Exemplo concreto de implementação de um modelo de ranking no LightGBM,
utilizando o objetivo 'lambdarank' para aprendizado de ranking.
Init docstring:
Inicializa o modelo com parâmetros padrão ou personalizados para ranking.

Args:
    params (dict, opcional): Parâmetros para o LightGBM. Caso não seja fornecido,
                             utiliza um conjunto padrão voltado para ranking.
    num_boost_round (int, opcional): Número de iterações (árvores) no treinamento.

In [ ]:
model.predict(client_features, news_features)

TypeError: predict() takes 2 positional arguments but 3 were given